# Testing s3

In [1]:
# imports
from importlib import reload
import os
os.environ['ENDPOINT_URL'] = 'https://s3.nautilus.optiputer.net'

import h5py 
import torch
from tqdm.auto import tqdm

from ulmo import io as ulmo_io
from ulmo.models import io as models_io
from ulmo.utils import HDF5Dataset

Cartopy not installed.  Some plots will not work!


# Load model

In [ ]:
pae = models_io.load_modis_l2()

# Load latents

In [ ]:
latent_file = '/home/jovyan/Oceanography/SST-OOD/Test/Evaluations/MODIS_R2019_2012_95clear_128x128_latents_std.h5'
#latent_h5.keys()
latent_h5 = h5py.File(latent_file, 'r')
latents = latent_h5['latents'][:]

In [ ]:
latents.shape

## Load LL

In [67]:
LL_file = '/home/jovyan/Oceanography/SST-OOD/Test/Evaluations/MODIS_R2019_2012_95clear_128x128_log_prob_std.h5'
LL_h5 = h5py.File(LL_file, 'r')
#LL_h5.keys()
LL = LL_h5['valid'][:]

In [68]:
LL[0]

44.542786

# Apply ulmo

## Data Loader

In [70]:
dset_l = torch.utils.data.TensorDataset(torch.from_numpy(latents).float())
loader = torch.utils.data.DataLoader(
    dset_l, batch_size=1024, shuffle=False, 
    drop_last=False, num_workers=16)

## Log prob

In [71]:
#with h5py.File(output_file, 'w') as f:
with torch.no_grad():
    log_prob = [pae.flow.log_prob(data[0].to(pae.device)).detach().cpu().numpy()
            for data in tqdm(loader, total=len(loader), unit='batch', desc='Computing log probs')]

Computing log probs:   0%|          | 0/714 [00:00<?, ?batch/s]

In [72]:
log_prob[0]

array([-2170.355 , -1502.8975, -1434.7456, ..., -3031.3735, -1349.7324,
       -1992.5619], dtype=float32)

In [20]:
data = [data for data in loader]

In [23]:
data[0][0].shape

torch.Size([1, 512])

In [10]:
embedded_contet = pae.flow.flow._embedding_net(None)

In [27]:
noise, logabsdet = pae.flow.flow._transform(data[0][0].to(pae.device))

In [29]:
noise.shape

torch.Size([1, 512])

In [35]:
logabsdet

tensor([-560.3973], device='cuda:0', grad_fn=<AddBackward0>)

In [31]:
embedded_contet

In [34]:
pae.flow.flow._distribution.log_prob(noise)

tensor([-1701.7292], device='cuda:0', grad_fn=<SubBackward0>)

In [33]:
noise

tensor([[ 4.2856, -2.9644,  4.9077, -5.2754,  2.3232, -1.0631,  3.2675, -1.4780,
          2.1608,  0.7712, -7.3900, -2.2922, -2.4882, -0.5822,  0.2374,  3.8715,
          0.0624,  1.0902, -0.4192, -0.6375, -0.1435,  0.1635, -0.0995,  1.3944,
         -1.2614, -0.5138,  0.6984, -1.4532, -0.8175, -1.0216,  0.5286, -0.8830,
          0.7977, -0.6997, -0.2825,  0.3575, -3.4833,  1.3123,  0.8045, -1.3445,
         -3.1661,  2.3120,  0.6416,  0.3080, -0.4698, -0.1220,  1.7378, -2.3613,
          3.6702,  0.9197,  1.5923, -1.3069, -3.1598,  1.6582,  4.3597, -4.4577,
          0.1569, -1.0372,  3.1883,  0.9863,  1.4218,  1.0572,  3.1231,  0.2645,
          1.6775,  0.5029, -5.8601, -1.0793,  2.5416, -5.8976, -0.3600, -0.8460,
          3.0256,  2.6811, -1.3420, -2.6690, -1.5031, -5.9358,  2.0237,  2.7620,
         -0.5121,  0.0279,  2.9817, -3.8670, -0.2974, -1.5294,  3.3040, -1.3577,
         -2.7910,  1.9800, -0.2363, -0.1808,  1.4362,  2.1273, -0.2986,  0.3392,
         -2.8925,  2.0380, -

----

# Testing

In [3]:
pp_file = '/home/jovyan/Oceanography/SST-OOD/Test/PreProc/MODIS_R2019_2012_95clear_128x128_preproc_std.h5'

In [4]:
dset = HDF5Dataset(pp_file, partition='valid')

In [5]:
latents, LL = pae.compute_log_probs(dset)

Calculating latents..


KeyError: Caught KeyError in DataLoader worker process 8.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/jovyan/Oceanography/python/ulmo/ulmo/utils/utils.py", line 29, in __getitem__
    data = self.h5f[self.partition][index]
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "/opt/conda/lib/python3.8/site-packages/h5py/_hl/group.py", line 264, in __getitem__
    oid = h5o.open(self.id, self._e(name), lapl=sel